参考: https://acro-engineer.hatenablog.com/entry/2020/12/15/120000

In [ ]:
import pandas as pd
import optuna
import xfeat

In [ ]:
df_train = pd.read_csv('./data/train.csv')
df_test = pd.read_csv('./data/test.csv')

In [ ]:
# 学習データとテストデータをまとめる
df_all = pd.concat([df_train, df_test], axis=0)
df_all

In [ ]:
# カテゴリデータを抽出する
xfeat.SelectCategorical().fit_transform(df_all)

In [ ]:
# 数値データを抽出する
xfeat.SelectNumerical().fit_transform(df_all)

In [ ]:
# 数値の型をカテゴリに変換すると、カテゴリ変数として認識される
df_all['Pclass'] = df_all['Pclass'].astype('category')
xfeat.SelectCategorical().fit_transform(df_all).columns

In [ ]:
# ラベルエンコーディング
# suffixに何か指定すると、元のデータも残る
encoder = xfeat.Pipeline([
	xfeat.SelectCategorical(exclude_cols=['Name', 'Ticket']),
	xfeat.LabelEncoder(output_suffix='_le'),
])

df_le = encoder.fit_transform(df_all)
df_le

In [ ]:
# 複数のカテゴリ変数を組み合わせる
encoder = xfeat.Pipeline([
	xfeat.SelectCategorical(exclude_cols=['Name', 'Ticket']),
	xfeat.ConcatCombination(output_suffix='_cc', r=2),
])

encoder.fit_transform(df_le)